This notebook implements and plots/animate a Diffusion Limited Aggregation Model

Over a 2-D $n\times n$ grid with diffusion simulated (by solving Laplace Equation with Successive over Relaxation method) from top ($c=1$) to bottom ($c=0$), we introduce an evergrowing 'sinkhole' (concentration is always null or does not interact with concentration outside the object) to our grid. This sinkhole grows by randomly, one-by-one, adding neighbouring cells to its body. The probability for a neighbour to become part of the sinkhole depends on their concentration (after SOR is solved at precision $10^{-5}$) and a parameter $\eta$ which can provide more or less weight to neighbours and their concentration.  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
from numba import njit

In [ ]:
@njit
def update_contour(candidate_loc, contour, domain):
    '''
    This model of DLA relies on two (n x n) matrices:
    domain : is where we solve the diffusion equation (through laPlace; SOR)
    contour : is where the cluster (1s in the contour matrix) and the neighbouring candidates (2s) are tracked and updated
    
    Given a chosen candidate to aggregate, this function updates contour and thus the cluster accordingly.
    '''
    # initialize
    n = len(contour)
    x = candidate_loc[0]
    y = candidate_loc[1]
    domain[x, y] = 0
    contour[x, y] = 1

    # we now create a list of neighbours which we will reduce to include only the ones which should be part of the new contour
    neighbours = []
    
    # I exempt the boundaries of the grid from being in the cluster for stability 
    if x > 0:
        neighbours.append([x-1, y])
    if y > 0:
        neighbours.append([x, y-1])
    if x < n-1:
        neighbours.append([x+1, y])
    if y < n-1:
        neighbours.append([x, y+1])
    # if the neighbours of our aggregated candidated are not in the cluster the are tracked (=2)
    for neigh in neighbours: 
        if contour[neigh[0], neigh[1]] != 1 and neigh[0] in range(1, n-1) and neigh[1] in range(1, n-1):
            contour[neigh[0], neigh[1]] = 2
    # I pull the updated list of neighbouring candidates
    candidates = [[i, j] for i in range(0,n) for j in range(0,n) if contour[i, j] == 2]
    return contour, candidates

def aggregate_candidate(candidates, domain, eta):
    '''
    Given the list "candidates" of coordinates of the neighbours of our cluster,
    this function computes the probability of candidates to be chosen (p = c_ij**eta/sum(c_ij**eta))
    and chooses one randomly, weighing each options probability p
    '''
    # create a list parallel to 'candidates' which includes each candidate's probability respectively
    cand_eta = [(np.abs(domain[i, j]))**eta for [i, j] in candidates]
    prob_cand = [c/np.sum(cand_eta) for c in cand_eta]
    # I choose and 'return' a candidate randomly, based on their and other's probability
    return candidates[np.random.choice(len(candidates), p = prob_cand)]

@njit
def sor_iteration(matrix, contour, w):
    '''
    This is one iteration of Successive over Relaxation method for solving laplace equation 
    with updated rules for elements in our cluster/sinkhole (when contour[i,j] == 1)
    '''
    n = len(matrix)
    next_matrix = np.copy(matrix)
        
    for j in range (1, n-1):
        # rule out if part of cluster/sinkhole
        if contour[j,0] == 1:
            next_matrix[j, 0] = 0
        else:
            # west boundary case where x = 0:
            next_matrix[j,0] = w/4 * (matrix[j,1] + matrix[j,-2] + matrix[j+1,0] + next_matrix[j-1,0]) + (1 - w) * matrix[j, 0]

        # non-boundary case 
        for i in range (1, n-1):
            # rule out if part of cluster/sinkhole
            if contour[j,i] == 1:
                next_matrix[j, i] = 0
            else:
                next_matrix[j,i] = w/4 * (matrix[j,i+1] + next_matrix[j,i-1] + matrix[j+1,i] + next_matrix[j-1,i]) + (1 - w) * matrix[j, i]

        # rule out if part of cluster/sinkhole
        if contour[j,-1] == 1:
            next_matrix[j, -1] = 0
        else:
            # east boundary case where x = n:
            next_matrix[j,-1] = w/4 * (matrix[j,1] + matrix[j,-2] + matrix[j+1,-1] + next_matrix[j-1,-1]) + (1 - w) * matrix[j, -1]

    return next_matrix

@njit
def sor(domain, contour, w, eps = 1e-5):
    '''
    this runs the above SOR iteration function until stability with precision eps = 1e-5
    '''
    # initialize our counter iter and set an initial value of convergence
    conv = 1
    iter = 0 # we track this for comparing omega values (which won't be done in this notebook)

    # run sor_iteration until it converges with precision 1e-5
    while conv > eps:
        next_domain = sor_iteration(domain, contour, w)
        diff = np.abs(next_domain - domain) 
        # absolute value because SOR can be unstable and produce negative values which breaks our aggregate function
        # doing this introduce an error no greater than the one due to SOR's unstability
        conv = np.max(diff)
        domain = next_domain
        iter += 1
    return next_domain, iter

def next_step(domain, contour, candidates, eta, w):
    '''
    This updates our domain with our evergrowing cluster (tracked in contour), I used this for the animation only
    '''
    # pick a 'winning' neighbouring candidate
    cand = aggregate_candidate(candidates, domain, eta)
    # update the contour accordingly
    contour, candidates = update_contour(cand, contour, domain)
    # solve the diffusion equation again with new cluster
    domain, _ = sor(domain, contour, w)
    return domain, contour, candidates

The following kernel produces a $N$ size cluster with a given $\eta$, solved by SOR with relaxation factor $\omega$ over a grid of size $n\times n$. It also plots the results once all aggregated candidates were added to the cluster.

In [ ]:
n = 100 # grid length (once squared)
eta = 1.5 # shape parameter
w = 1.7 # relaxation factor
N = 400 #size of constructed cluster  

# Initialize domain and contour
domain = np.zeros((n, n))
domain[0, :] = 1
contour = np.zeros((n, n))
# initialize a 1x1 cluster at the bottom/middle of the grid
contour, candidates = update_contour([n-2, n//2], contour, domain)
domain, iter = sor(domain, contour, w)

for _ in range(N):
    # pick a 'winning' neighbouring candidate
    cand = aggregate_candidate(candidates, domain, eta)
    # update the contour accordingly
    contour, candidates = update_contour(cand, contour, domain)
    # solve the diffusion equation again with new cluster
    domain, _ = sor(domain, contour, w)


# I change the resulting domain a little so the color map shows more contrast, thus we see the cluster better
color_domain = np.copy(domain)
for x in range(0, n):
    for y in range(0, n):
        if contour[x, y] == 1:
            color_domain[x, y] = -1

# plot the resulting grid and cluster
im = plt.imshow(color_domain, cmap='hot_r')
plt.xlabel(f"x ({n})")
plt.ylabel(f"y ({n})")
plt.title(fr"Diffusion Limited Aggregation with $\omega=${w} and $\eta=${eta}, cluster of size {N}")
plt.xlim(1, n-1)
plt.ylim(n-1, 1)
#plt.savefig("dla_set2_2a.png", dpi = 300)
plt.show()

The following kernel animates the iterated growth of a cluster given apposite parameters. 

In [ ]:
n = 100 # grid length (once squared)
eta = 1 # shape parameter
w = 1.7
N = 400

# Initialize domain and contour
domain = np.zeros((n, n))
domain[0, :] = 1
contour = np.zeros((n, n))
contour, candidates = update_contour([n-2, n//2], contour, domain)
domain, iter = sor(domain, contour, w)

fig, ax = plt.subplots()
im = ax.imshow(color_domain, cmap='hot_r', animated=True)

ax.set_xlabel(f"x ({n})")
ax.set_ylabel(f"y ({n})")
ax.set_title(fr"Diffusion Limited Aggregation with $\omega=${w} and $\eta=${eta}")

def update(frame):
    global domain, contour, candidates
    domain, contour, candidates = next_step(domain, contour, candidates, eta, w)
    color_domain = np.copy(domain)
    for x in range(0, n):
        for y in range(0, n):
            if contour[x, y] == 1:
                color_domain[x, y] = -10
    im.set_array(color_domain)
    return [im]


ani = animation.FuncAnimation(fig, update, frames=N, interval=10, blit=False)

HTML(ani.to_jshtml())

For the rest of the notebook, I run the same model while trying to improve on the time required to solve the diffusion equation.
The idea is to use a multigrid: the smaller (with step size 1/n) is used just like we did previously. A bigger grid (step size a/n for a>1>n) is used to solve the diffusion equation **above** the cluster.

We need to track the height of the cluster
and update sor_iteration() function 

In [ ]:
@njit
def height_of_contour(contour):
    '''
    I measure and return the distance between the top of the grid and the top of the cluster
    '''
    height = -1
    for i in range(len(contour)):
        if 1 not in contour[i,:]:
            height += 1
        else:
            break
    return height

def improved_sor_iteration(domain, contour, w, new_step_size):
    '''
    This one is a little bulky, probably not optimal. 
    In this function, I create a coarser grid with a bigger step size.
    I first solve diffusion with sor for the coarse grid before it hits the object 
    (so only for the portion of the grid above the heighest point of the cluster)
    I then spread this solution to the finer grid, 
    where every fine step takes the value of whichever coarse step they are in.
    '''
    height = height_of_contour(contour)
    
    # We rule out a new step size too large
    if new_step_size > height:
        return f'The given step size is too big'
    
    # set up dimensions of new/coarser grid
    new_grid_size = height // new_step_size
    n = len(domain[0])
    new_grid = np.zeros((new_grid_size, n // new_step_size))
    
    init = [] #this has the unique task of being a 1. list above the coarse grid to allow proper solving
    fin = [] #same with 0. under the coarse grid
    
    # let us create the new grid from average of corresponding finer grid 
    for j in range(n // new_step_size):
        init.append(1.) 
        fin.append(0.)
        
        for i in range(1, new_grid_size):
            sub_grid = [domain[h, k] for h in range(i * new_step_size, (i + 1) * new_step_size) 
                                       for k in range(j * new_step_size, (j + 1) * new_step_size)]
            new_grid[i, j] = np.mean(sub_grid)
    
    # Prepare new grid by inserting boundary conditions
    new_grid_prep = np.copy(new_grid)
    new_grid_prep = np.insert(new_grid_prep, 0, init, axis=0)
    new_grid_prep = np.append(new_grid_prep, [fin], axis=0)
    
    # Solve the new coarse grid
    new_contour = np.zeros_like(new_grid_prep)
    new_grid_solved, new_iter = sor(new_grid_prep, new_contour, w)
    new_grid_solved = new_grid_solved[1:-1]  # Remove the added boundary rows (init and fin)
    
    # Map the solved coarse grid back to the fine grid
    for j in range(n // new_step_size):
        for i in range(1, new_grid_size):
            domain[i * new_step_size:(i + 1) * new_step_size, j * new_step_size:(j + 1) * new_step_size] = new_grid_solved[i, j]
    
    return domain, new_iter


def improved_sor(domain, contour, w, new_step_size):
    '''
    This is a certain sequence for multigrid solving
    I would have tested more but this one served to improve on regular sor.
    1. Solve the coarse grid above object
    2. Solve the finer grid for the whole domain
    end
    '''
    #Solve coarser grid
    domain, new_iter = improved_sor_iteration(domain, contour, w, new_step_size)
    
    #solve finer grid
    domain, iter = sor(domain, contour, w)
    
    return domain, iter + new_iter


In [ ]:
def improved_sor_iteration_2(domain, contour, w, new_step_size):
    '''
    this is the last version of SOR with multi grid solved over the whole domain (not only above the cluster)
    On the coarser grid, any coarse step which contains any finer step part of the cluster, is consider part of the cluster when solving coarse grid 
    '''
    n = len(domain[0])
    
    # We rule out a new step size too large
    # set up dimensions of new/coarser grid
    new_grid_size = n // new_step_size
    new_grid = np.zeros((new_grid_size, n // new_step_size))
    new_contour = np.zeros((new_grid_size, n // new_step_size))
    
    init = [] #this has the unique task of being a 1. list above the coarse grid to allow proper solving
    fin = [] #same with 0. under the coarse grid
    
    # let us create the new grid from average of corresponding finer grid 
    for j in range(new_grid_size):
        init.append(1.) 
        fin.append(0.)
        
        for i in range(1, new_grid_size-1):
            sub_grid = [domain[h, k] for h in range(i * new_step_size, (i + 1) * new_step_size) 
                                       for k in range(j * new_step_size, (j + 1) * new_step_size)]
            contour_sub_grid = [contour[h, k] for h in range(i * new_step_size, (i + 1) * new_step_size) 
                                       for k in range(j * new_step_size, (j + 1) * new_step_size)]
            if np.any(contour_sub_grid == 1):
                new_contour[i, j] = 1
                new_grid[i, j] = 0
            else:
                new_grid[i, j] = np.mean(sub_grid)
    
    # Prepare new grid by inserting boundary conditions
    new_grid_prep = np.copy(new_grid)
    new_grid_prep = np.insert(new_grid_prep, 0, init, axis=0)
    new_grid_prep = np.append(new_grid_prep, [fin], axis=0)
    
    # Solve the new coarse grid
    new_grid_solved, new_iter = sor(new_grid_prep, new_contour, w)
    new_grid_solved = new_grid_solved[1:-1]  # Remove the added boundary rows (init and fin)
    
    # Map the solved coarse grid back to the fine grid
    for j in range(new_grid_size):
        for i in range(1, new_grid_size-1):
            domain[i * new_step_size:(i + 1) * new_step_size, j * new_step_size:(j + 1) * new_step_size] = new_grid_solved[i, j]
    
    return domain, new_iter


def improved_sor_2(domain, contour, w, new_step_size):
    '''
    This is a certain sequence for multigrid solving
    1. Solve the coarse grid
    2. Solve the finer grid for the whole domain
    end
    '''
    #Solve coarser grid
    domain, new_iter = improved_sor_iteration_2(domain, contour, w, new_step_size)
    
    #solve finer grid
    domain, iter = sor(domain, contour, w)
    
    return domain, iter + new_iter


In [ ]:
n = 200 # grid length (once squared)
eta = 1.5 # shape parameter
w = 1.7 # relaxation factor
N = 400 #size of constructed cluster  

# Initialize domain and contour
domain = np.zeros((n, n))
domain[0, :] = 1
contour = np.zeros((n, n))
# initialize a 1x1 cluster at the bottom/middle of the grid
contour, candidates = update_contour([n-2, n//2], contour, domain)
domain, iter = sor(domain, contour, w)

for _ in range(N):
    # pick a 'winning' neighbouring candidate
    cand = aggregate_candidate(candidates, domain, eta)
    # update the contour accordingly
    contour, candidates = update_contour(cand, contour, domain)
    # solve the diffusion equation again with new cluster
    domain, _ = sor(domain, contour, w)


# I change the resulting domain a little so the color map shows more contrast, thus we see the cluster better
color_domain = np.copy(domain)
for x in range(0, n):
    for y in range(0, n):
        if contour[x, y] == 1:
            color_domain[x, y] = -1

# plot the resulting grid and cluster
im = plt.imshow(color_domain, cmap='hot_r')
plt.xlabel(f"x ({n})")
plt.ylabel(f"y ({n})")
plt.title(fr"Diffusion Limited Aggregation with $\omega=${w} and $\eta=${eta}, cluster of size {N}")
plt.xlim(1, n-1)
plt.ylim(n-1, 1)
plt.show()

In [ ]:
# Test for sor solution to diffusion
domain = np.zeros((n, n))
domain[0, :] = 1
domain, iter = sor(domain, contour, w)
print(iter, domain)

In [ ]:
# Test for improved sor solution to diffusion
domain = np.zeros((n, n))
domain[0, :] = 1
domain, iter = improved_sor(domain, contour, w, new_step_size=10)
print(iter, domain)

In [ ]:
# Test for second improved sor solution to diffusion
domain = np.zeros((n, n))
domain[0, :] = 1
domain, iter = improved_sor_2(domain, contour, w, new_step_size=10)
print(iter, domain)

These three tests may not show any big improvements
In fact, for some values of 'new_step_size', the multigrid process requires more iterations. But I ran these tests for $n=500$, with some $400$ sized cluster:
- SOR required $16933$ iterations
- Improved_SOR required $14202$ iterations(new_step_size $=10$)
- Improved_SOR_2 required $1881$ iterations(new_step_size $=10$)

You can test this (5 min) by changing $n$ to $500$ in the kernel above the 3 tests and rerunning them.
I didn't test this too much but the multigrids definitely improve on sor solution.